# Generating mutually exclusive n-hot  coding

Suppose the number of categories is $C$  and number of output neurons is $m$  ($ n \cdot C \leq m$). For generating mutually exclusive $n$-hot code vectors of size $m$ for each category, we started from the first category to the last one and successively for each category $c \in \{0,1,\cdots,C-1\}$ we initialized its code vector with zero elements and then randomly selected $n$ out of $m-c \cdot n$ elements that were not equal to $1$ in any of the $c$ previously coded category vectors and set them equal to $1$. 

In [ ]:
import random
import numpy as np
import torch

dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




In [ ]:
def Diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

# coding_layers should be a list of number of neurons in each layer 
# ones_in_layes should be a list of number of ones in each coding vector
 
def get_n_hot_coding_map(coding_layers , ones_in_layes   , number_of_categories  ):

  if(type(coding_layers) != list ):
    raise Exception('coding_layers is not a list')
  if(type(ones_in_layes) != list ):
    raise Exception('ones_in_layes is not a list')


  if(type(number_of_categories) != int ):
    raise Exception('number_of_categories is not int')      

  if(  len(coding_layers) != len(ones_in_layes)  ):
    raise Exception('inputs len mismatch')  


  coding = []

  


  for i in range(len(coding_layers)):
    if ( number_of_categories*(ones_in_layes[i]  ) > coding_layers[i]  ):
      raise Exception('no a valide coding')



    coding.append( np.zeros([number_of_categories,coding_layers[i]])     )

    initial_indices_list = list(range(coding_layers[i]))
    for j in range(number_of_categories):

      indice = random.sample( initial_indices_list  , ones_in_layes[i] )
      coding[i][j,indice] = 1
      k=0
      initial_indices_list = Diff(initial_indices_list, indice)
        


    coding[i] = torch.tensor(coding[i] , device=device, dtype=dtype, requires_grad=False )
  
  return coding

# print(device)

# coding  = get_n_hot_coding_map([100] , [ 10 ]       , 10 )    
# # # print(coding)

# m =  np.zeros( len(coding[0][0]) )

# for i in range(len(coding[0])):
#   m = m + np.array(coding[0][i])

# print(m)

# m = np.ones(len(coding[0][0]))
# for i in range(len(coding[0])):
#   m = m * np.array(coding[0][i])

# print(m)

# # print( np.array(coding[1][0]) + np.array(coding[1][1]) + np.array(coding[1][2]) )
# # print( np.array(coding[1][0]) * np.array(coding[1][1]) * np.array(coding[1][2]) )
# # print( np.array([0,2,3]) * np.array([3,2,5])  )
# # i=0

# # print(  torch.matmul(   coding[i]  , torch.transpose(coding[i],0, 1)      )   )



# # i=1

# # print(  torch.matmul(   coding[i]  , torch.transpose(coding[i],0, 1)      )   )

In [ ]:
# coding  : list of tensors with size : ( number_of_categories , coding_layers[i])   ,  coding_layers[i] is number of neurons in layer i
# category : 1:can be (Batch size , 1)  2:can be (Batch size  ) 


def code_category(coding , category):
  if(type(coding) != list ):
    raise Exception('coding is not a list')

  if(not torch.is_tensor(category) ):  
    raise Exception('category is not a torch tensor')

  coded = []
  
  if (len(category.shape)  == 1  or  (len(category.shape)  == 2  and  (category.shape[1])  == 1)):

    category1 = category.view(-1).to(torch.int64 )
    for i in range( len(coding) ):
      coded.append(coding[i][  category1   , :])
    
    
  #if its one hot code
  else :
    raise Exception('category size mismach')

  return coded 



    # for i in range(len(coding)):
    #   output.append()

# a=np.array(  [2,1,1,1,1,1,1])
# # print(a.shape)
# category = torch.tensor(a).view([-1,1])

# category = torch.tensor([ [0,0,1] ,  [1,0,0]  ,  [1,0,0]  ,  [1,0,1] ] , dtype=dtype )
# # category = torch.tensor(a)
# # category = a
# # print(category.shape)
# print(category)
# # print(category.shape)


# code = get_distributed_coding_map([5,3] , [ 2,1]    , [1,0]   , 3 ) 
# print(code)

# code_category(code , category)

In [ ]:
# coding  : list of tensors with size : ( number_of_categories , coding_layers[i])   ,  coding_layers[i] is number of neurons in layer i
# activity  : list of activity of layers with size : ( batch_size , coding_layers[i])

# return  : tesor of size : ( batch_size , top )

def decode_category(coding , activity , top=1 , coef=None):
  if(type(coding) != list ):
    raise Exception('coding is not a list')

  if(type(activity) != list ):
    raise Exception('activity is not a list')

  result =  torch.zeros( [  activity[0].shape[0] , coding[0].shape[0] ]  , device = device , dtype=dtype, requires_grad=False )


  for i in range(len(coding)):
    if (coef==None):
      result = result + torch.matmul( activity[i]  , torch.transpose( coding[i] , 0 , 1 ) ) 

    else:
      result = result + coef[i] * torch.matmul( activity[i]  , torch.transpose( coding[i] , 0 , 1 ) ) 

  result2 =  torch.zeros( [activity[0].shape[0] , top ] , device = device , dtype=dtype, requires_grad=False )


  for i in range(top): 
    a,indece = torch.max(result ,dim=1)

    result2[np.arange(activity[0].shape[0])  , i ] = indece.view([-1]).to(dtype)
    result[np.arange(result.shape[0]) ,indece.view([-1]) ] = -1
  return result2
  


# code = get_distributed_coding_map([5,3] , [ 2,1]    , [1,0]   , 3 ) 

# print(code[0])
# print(code[1])

# code1 =  torch.tensor( [[1,0,0,1,0] , [0,1,0,0,0] , [0,0,1,0,0]  ] )
# code2 =  torch.tensor( [[1,0,0] , [0,1,0] , [0,0,1]  ] )



# activity1 = torch.tensor( [[1,1,1,1,0] , [0,1,0,1,0] , [0,0,0,1,0] , [0,0,1,0,0] , [0,0,1,0,0] , [0,1,0,1,0]] )

# activity2 = torch.tensor( [[1,0,1] , [0,0,0] , [0,1,0] , [0,0,1] , [0,1,0] , [0,1,0]] )

# res = decode_category([code1,code2] , [activity1,activity2] ,top=2 , coef = [0.1 , 0.9]  )

# print(res)

In [ ]:

# true_labels size : (batch size , 1)  or (batch size)
def get_accuracy( true_labels , coding  , activity , top=1 , coef=None ):

  if(type(activity) != list ):
    raise Exception('activity is not a list')

  if(true_labels.shape[0] != activity[0].shape[0]  or  len(true_labels.shape) >  1):
    raise Exception('true_labels shape mismatch')

  if(not torch.is_tensor(true_labels) ):  
    raise Exception('true_labels is not a torch tensor')

  true_labels = true_labels.view([-1,1])


  predicted = decode_category(coding , activity , top=top , coef=coef)

  res = torch.sum(torch.clamp( torch.sum(predicted == true_labels , dim=1)  , 0 ,1 )).to(dtype)/true_labels.shape[0]
  return res.item()



# code1 =  torch.tensor( [[1,0,0,1,0] , [0,1,0,0,0] , [0,0,1,0,0]  ] )
# code2 =  torch.tensor( [[1,0,0] , [0,1,0] , [0,0,1]  ] )



# activity1 = torch.tensor( [[1,1,1,1,0] , [0,1,0,1,0] , [0,0,0,1,0] , [0,0,1,0,0] , [0,0,1,0,0] , [0,1,0,1,0]] )

# activity2 = torch.tensor( [[1,0,1] , [0,0,0] , [0,1,0] , [0,0,1] , [0,1,0] , [0,1,0]] )

# true_labels = torch.tensor([1,1,0,2,1,2])
# get_accuracy( true_labels  , [code1,code2] , [activity1,activity2] ,top=2)


In [ ]:
# x list of size (batch , coding[0].shape[1]  + coding[1].shape[1]  + ... )

def seprate(coding , x):
  if(not torch.is_tensor(x) ):  
    raise Exception('x is not a torch tensor')

  if(type(coding) != list ):
    raise Exception('coding is not a list')

  mlist = []
  form_ = 0
  til=0
  for i in range(len(coding)):
    til = til + coding[i].shape[1]
    mlist.append(x[: , form_ : til])
    form_ = form_ + coding[i].shape[1]
    if(til > x.shape[1]):  
      raise Exception('x size mismatch')


  if(til != x.shape[1]):  
    raise Exception('x size mismatch')

  return mlist


# coding  = get_distributed_coding_map([3,4] , [ 1 ,1 ]    , [2,1]   , 3 )    

# x = torch.rand([5,7])

# y =seprate(coding , x)
# print(x)
# print(y)

# x = torch.tensor([[1,2,3.3],[5.4,1,0.2]])
# print(x)
# a,b = torch.max(x ,dim=1)

# x[np.arange(x.shape[0]) , b ] = -1
# print(b)

# print(x)

# a,b = torch.max(x ,dim=1)

# x[np.arange(x.shape[0]) , b ] = -1
# print(b)

# print(x)